### Installing all necessary dependencies

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!python -m spacy download es_core_news_lg

### Import the dependencies

In [ ]:
from unsloth import FastLanguageModel
import torch
import spacy

### NER for preprocessing the news to recover the location

In [ ]:
nlp = spacy.load("es_core_news_lg")

In [ ]:
def extract_first_loc(text):
    doc = nlp(text)
    first_loc = next((ent.text for ent in doc.ents if ent.label_ == "LOC"), None) # Find the first entity of type LOC

    return first_loc

### Processing the news and generation function


In [ ]:
def generate_llama3(noticia, fecha):

  loc = extract_first_loc(noticia)
  if(loc):
          input = "Crea un artículo de noticias con esta información: " + noticia + ". " + "Fecha: " + fecha + "." + "Lugar: " + loc + "."
  else:
          input = "Crea un artículo de noticias con esta información: " + noticia + ". " + "Fecha: " + fecha

  alpaca_prompt = """Tu tarea es escribir artículos de noticia que contengan siempre una fecha, un lugar y un acontecimiento. No puedes inventar información que no se te da, utiliza lenguaje formal.

  ### Instruction:
  {}

  ### Input:
  {}

  ### Response:
  {}"""

  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "Crea un artículo de noticias con esta información:", # instruction
          input, # input
          "",
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)

  text = tokenizer.batch_decode(outputs)

  # Convertir lista a cadena (si hay más de un elemento)
  text = " ".join(text)

  # Utilizar una expresión regular para encontrar el texto después de '### Response:'
  response_text = re.search(r'### Response:\s*(.*)', text, re.DOTALL)

  # Si se encuentra la coincidencia, imprimir el texto
  if response_text:
      return  response_text.group(1).strip()
  else:
      return "No se encontró '### Response:' en el texto."

### Loading the model

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "BrauuHdzM/llama3_noticias_espanol",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

FastLanguageModel.for_inference(model)

### Generate the news

In [ ]:
# Give information about the news. This is an example!
noticia = "Melbourne. El español Carlos Sainz (Ferrari), ganador del Gran Premio de Australia de Fórmula Uno, aseguró que confiaba en tener la capacidad de superar al tricampeón mundial y actual líder de la competencia, el neerlandés Max Verstappen (Red Bull), hoy en la tercera prueba de la temporada, e indicó que su triunfo demuestra que nunca hay que darse por vencido."
fecha = "25 de marzo del 2024"

In [ ]:
generate_llama3(noticia, fecha)